# Ruta 2 (Baldwin-lite) - Entrenamiento en Colab (GPU L4)

Este notebook ejecuta `entrenar_pipeline_completo.py` usando GPU (idealmente L4).

## Requisitos
- Tener el repo en GitHub **o** subir la carpeta `cDAQ_9174` a Google Drive.
- En Colab: `Runtime -> Change runtime type -> GPU`.


In [ ]:
!nvidia-smi
import torch
print('torch', torch.__version__)
print('cuda available', torch.cuda.is_available())
if torch.cuda.is_available():
    print('device', torch.cuda.get_device_name(0))


## Opción A: clonar repo (recomendado)
Edita `REPO_URL` con tu URL (HTTPS) y ejecuta.


In [ ]:
# @title Clone repo levitador-benchmark
REPO_URL = 'https://github.com/JRavenelco/levitador-benchmark.git'
BRANCH = 'main'

!rm -rf levitador-benchmark
!git clone --depth 1 -b {BRANCH} {REPO_URL} levitador-benchmark

%cd levitador-benchmark
!ls

## Opción B: usar Google Drive (si no quieres clonar)
Sube la carpeta `cDAQ_9174` a tu Drive, luego ajusta `DRIVE_PROJECT_DIR`.


In [ ]:
# @title Mount Drive (opcional)
from google.colab import drive
drive.mount('/content/drive')

DRIVE_PROJECT_DIR = ''  # e.g. /content/drive/MyDrive/cDAQ_9174
if DRIVE_PROJECT_DIR:
    %cd $DRIVE_PROJECT_DIR
    !ls


In [ ]:
# Instalar dependencias mínimas
!python -m pip install -q tqdm pandas matplotlib


In [ ]:
# Entrenar (maximizando GPU L4 / A100)
# La L4 tiene 22.5GB VRAM - podemos usar batches MUCHO más grandes

import os

os.environ["USE_AMP"] = "1"
os.environ["NUM_WORKERS"] = "4"

# Más épocas para generalización multi-sesión (Baldwin-lite)
os.environ["EPOCHS_STAGE1"] = "1200"
os.environ["EPOCHS_STAGE2"] = "2400"

# Batch agresivo para saturar la GPU (L4 aguanta 4096+ fácil con este modelo)
os.environ["BATCH_SIZE"] = "4096"

print(f"Config: BATCH={os.environ['BATCH_SIZE']}, EPOCHS1={os.environ['EPOCHS_STAGE1']}, EPOCHS2={os.environ['EPOCHS_STAGE2']}, AMP={os.environ['USE_AMP']}")

!python scripts/entrenar_ruta2.py

## Salidas
Al terminar, quedan en `levitador-benchmark/`:
- `flux_observer_final.pt`
- `position_predictor_final.pt`
- `sensorless_pipeline_final.pt`

Descarga los `.pt` o cópialos a Drive para usarlos en el controlador C++.